In [1]:
TRAIN_ESSAYS_PATH = '/kaggle/input/llm-detect-ai-generated-text/train_essays.csv'
TEST_ESSAYS_PATH = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv'
TRAIN_PROMPTS_PATH = '/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv'

MODEL_PATH = '/kaggle/input/bertbaseconstantlearningerrado'
# MODEL_PATH = '/kaggle/input/d/faguim/modelo3'
# MODEL_PATH = '/kaggle/input/bertlarge2eminus5-zip'
# MODEL_PATH = '/kaggle/working/output_dir/checkpoint-7728'

# MODEL_PATH = 'bert-base-cased'

In [2]:
# !ls -lh /kaggle/input

In [3]:
import pandas as pd

# train_essays = []
t_essays = pd.read_csv(TRAIN_ESSAYS_PATH)
test_essays = pd.read_csv(TEST_ESSAYS_PATH)
train_prompts = pd.read_csv(TRAIN_PROMPTS_PATH)

ext_df1 = pd.read_csv('/kaggle/input/propertrain-dataset/train_drcat_01.csv')
ext_df2 = pd.read_csv('/kaggle/input/propertrain-dataset/train_drcat_02.csv')
ext_df3 = pd.read_csv('/kaggle/input/propertrain-dataset/train_drcat_03.csv')
ext_df4 = pd.read_csv('/kaggle/input/propertrain-dataset/train_drcat_04.csv')

t_essays['label'] = t_essays.generated.copy()

ext_df = pd.concat([
    t_essays,
    ext_df1,
    ext_df2,
    ext_df3,
    ext_df4,
])

ext_df.drop(['id', 'essay_id', 'source', 'prompt', 'prompt_id', 'fold', 'generated'], axis='columns', inplace=True)
ext_df.drop_duplicates(keep='first', inplace=True)
print(ext_df)

train_essays = ext_df.copy().reset_index(drop=True)

                                                    text  label
0      Cars. Cars have been around since they became ...      0
1      Transportation is a large necessity in most co...      0
2      "America's love affair with it's vehicles seem...      0
3      How often do you ride in a car? Do you drive a...      0
4      Cars are a wonderful thing. They are perhaps o...      0
...                                                  ...    ...
44144  \nSeeking advice from more than one person whe...      1
44160  While the Facial Action Coding System technolo...      1
44162  I strongly encourage you to participate in the...      1
44163  Dear Principal Smith,\n\nI am writing to you r...      1
44197  The Seagoing Cowboys program is an amazing opp...      1

[44155 rows x 2 columns]


In [4]:
# !pip install --upgrade -q wandb git+https://github.com/soumik12345/wandb-addons

# import wandb  # Import wandb library for experiment tracking
# import wandb_addons # Additional wandb utilities

# try:
#     from kaggle_secrets import UserSecretsClient  # Import UserSecretsClient
#     user_secrets = UserSecretsClient()  # Create secnovo3 = test_dataset2.map(tokenize)rets client instance
#     api_key = user_secrets.get_secret("WANDB")  # Get API key from Kaggle secrets
#     wandb.login(key=api_key)  # Login to wandb with the API key
#     anonymous = None  # Set anonymous mode to None
#     wandb.init() 
#     wandb.log()
# except:
#     anonymous = 'must'  # Set anonymous mode to 'must'
#     wandb.login(anonymous=anonymous, relogin=True)  # Login to wandb anonymously and relogin if needed

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

print('Dispositivo:', device)
torch.cuda.get_device_name(device)

Dispositivo: cuda


'Tesla P100-PCIE-16GB'

In [6]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForSequenceClassification,  
    EvalPrediction,
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, max_length=512, model_max_length=512)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, max_length=1024, model_max_length=1024)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, max_length=1024)
# tokenizer = AutoTokenizer.from_pretrained(BERT_PATH, max_length=1024)

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
from datasets import Dataset 

train_dataset = Dataset.from_dict(train_essays)
tokenized_train_dataset = train_dataset.map(tokenize, batched=True)

train_split, eval_split, test_split = torch.utils.data.random_split(tokenized_train_dataset, [0.7,0.15, 0.15])

  0%|          | 0/45 [00:00<?, ?ba/s]

In [9]:
# len(train_split[100]["input_ids"])

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_PATH,
#         BERT_PATH,
        num_labels=2,
#         max_position_embeddings=512,
#     ignore_mismatched_sizes=True
#         use_safetensors=True,
    )

training_args = TrainingArguments(
        output_dir="output_dir",
#         eval_delay=0,
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        weight_decay=3e-6,
        adam_beta1=0.9,
        adam_beta2=0.999,
        adam_epsilon=1e-8,
#     num_cycles=3,
        num_train_epochs=5,
        lr_scheduler_type="constant",
        warmup_steps=579,
        log_level="info",
        save_strategy="epoch",
        save_total_limit = 5,
#         load_best_model_at_end = True,
#         seed=43,
        fp16=True,
#         dataloader_drop_last=True,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,

    )

trainer = Trainer(
        model,
        training_args,
        tokenizer=tokenizer,
#         data_collator=collator,cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)
        train_dataset=train_split,
        eval_dataset=eval_split
    )

Using auto half precision backend


In [11]:
# trainer.train()
# trainer.save_model()

------------------------------------------------
# Teste

In [12]:
# result = trainer.evaluate(test_split)
# result

In [13]:
# os.environ["WANDB_DISABLED"] = "true"

In [14]:
# result

In [15]:
# # # # 1: 'eval_loss': 0.018171824514865875,
# # # # 2:'eval_loss': 0.019168734550476074
# # # # 3: 'eval_loss': 0.016240527853369713'
        
        
        
# # # #         ---------------
# # # #     Modelo 4    
# # # # Epoch 	Training Loss 	Validation Loss --> Test
# # # # 1 	0.038200 	0.133390
# # # # 2 	0.029200 	0.043577
# # # # 3 	0.013600 	0.039176 ---> 0.037069786339998245

# # # 10EPOCHS
# # # 1 	0.040600 	0.033435
# # # 2 	0.016000 	0.035558
# # # 3 	0.010200 	0.031337
# # # 4 	0.012400 	0.043263
# # # 5 	0.006100 	0.041467
# # # 6 	0.004400 	0.039544
# # # 7 	0.003000 	0.048936
# # # 8 	0.000000 	0.045338
# # # 9 	0.000000 	0.046916
# # # 10 	0.000000 	0.047661

# # # 7EPOCHS
# # # 1 	0.032500 	0.044884
# # # 2 	0.016900 	0.019400
# # # 3 	0.005200 	0.038988  0.015834501013159752,
# # # 4 	0.004300 	0.020831  0.00468977726995945
# # # 5 	0.003600 	0.045806  0.017297109588980675
# # # 6 	0.000100 	0.037682  0.013847769238054752
# # # 7 	0.000000 	0.031262  0.013523745350539684


# # # 5epochslarge, lr 3e-5
# # # 5 0.6 --> 0.6 --> 0.6283125877380371


# # # 2epochslarge, lr 2e-5
# # # 1 	0.032600 	0.015285   0.00930606760084629,
# # # 2 	0.007400 	0.011398   0.018849071115255356

# # # ---------------------------
# # # 1epochslarge, lr 3e-5
# # # x --> x --> test_loss': 0.15940824151039124,


        
    
# # bert-base learning_rate=3e-5, warmup steps desconfigurado
# # 1 	0.035900 	0.020002   0.011627799831330776,
# # 2 	0.014300 	0.026277   0.046091921627521515
# # 3 	0.004200 	0.021399   0.026681404560804367



# # bert-base learning_rate=3e-5, warmup steps 10%
# # 0.01594146341085434
# # 0.023494239896535873
# # 0.0223104078322649



# # bert-base learning_rate=2e-5, warmup steps 10%

# 1 	0.028800 	0.011249   0.006494616623967886
# 2 	0.015200 	0.013231   0.004830355755984783
# 3 	0.006000 	0.017011   0.02094988152384758


# bert-base 5e-5  warmup steps 10%
# 1 	0.038900 	0.028577  0.012289708480238914
# 2 	0.017700 	0.024407  0.01493032556027174
# 3 	0.007900 	0.045755  0.030657973140478134
# 4 	0.005000 	0.023188
# 5 	0.003300 	0.023409   0.023820417001843452



# bert-large 2e-5  warmup steps 10%

# 1 	0.057900 	0.050056
# 2 	0.031500 	0.028740
# 3 	0.017000 	0.016659
# 4 	0.015300 	0.019737
# 5 	0.011300 	0.021252  0.011726350523531437,


# bert-base constant-learning   5 epochs warmup_steps=579 (deveria ter 10%, taerrado)
# 1 	0.036700 	0.030619    0.02572002448141575
# 2 	0.022700 	0.014526    0.0069252680987119675
# 3 	0.020000 	0.017621    0.01026342622935772
# 4 	0.005300 	0.024898    0.009119929745793343
# 5 	0.005100 	0.022127 --> 0.01931276172399521

----------------------------------------------
# Submission

In [16]:
from datasets import Dataset 
from transformers import pipeline

test_dataset = Dataset.from_dict(test_essays)
test_dataset.map(tokenize)

# classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0, batch_size=16)

resultado = classifier(test_dataset[:]['text'], padding=True, truncation=True, max_length=512)

# resultado = classifier(test_dataset[:]['text'], padding=True, truncation=True, max_length=1024)
resultado

  0%|          | 0/3 [00:00<?, ?ex/s]

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_0', 'score': 0.9928627610206604},
 {'label': 'LABEL_0', 'score': 0.9837666749954224},
 {'label': 'LABEL_0', 'score': 0.9897630214691162}]

In [17]:
import csv 

with open('submission.csv', "w") as file:
    i = 0
    writer = csv.writer(file, dialect="excel")
    writer.writerow(["id","generated"])

    for p in resultado:        
        writer.writerow([test_dataset[i]['id'], p['score']])
        i = i+1

In [18]:
# !zip -r bertbaseconstantlearningerrado.zip /kaggle/working/output_dir/checkpoint-3864

# from IPython.display import FileLink
# FileLink(r'bertbaseconstantlearningerrado.zip')

In [19]:
# !rm -r output_dir/

In [20]:
# # !rm -r wandb/
# !rm -r /kaggle/input/d/faguim/modelo3

In [21]:
# !ls output_dir/ -lh

In [22]:
# import os
# os.remove("state.db")